In [30]:
import os
import torch
import logging
import torch.nn as nn
import numpy as np
from torchvision import transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from Code.DataSet.Preprocess import seqence_image
from Code.Model.VIT import get_model

In [31]:
img_path = './Examples/test1.JPEG'
img_size = 224
to_size = (8, 8, 3)
num_patches = 196

out, _ = seqence_image(img_path, img_size, to_size, num_patches)

In [32]:
print(out.shape)

(8, 1568, 3)


In [33]:
class ImageNetDataset(Dataset):
    def __init__(self, to_size=(8, 8, 3), num_patches=196,transform=None):
        self.transform = transform
        self.img_size = img_size
        self.to_size = to_size
        self.num_patches = num_patches

    def __len__(self):
        return 60000

    def __getitem__(self, idx):
        img = np.random.rand(8, 1568, 3) 
        
        img = (img * 255).astype(np.uint8)
        
        if self.transform:
            img = self.transform(img)
        
        img = torch.tensor(img, dtype=torch.float32)
        
        return img, 0

In [34]:
def learn(model, dataloader, num_epochs, optimizer, criterion, scheduler, device):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(dataloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if (i % 10 == 0):
                running_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                running_loss = 0.0

        accuracy = 100 * correct / total
        logging.info(f"[Epoch {epoch + 1}] Acc: {accuracy:.2f}%")

        scheduler.step()

    return model

In [35]:
def train(model_type = 'vit_base_patch16_224',
         batch_size = 128, img_size = 224, num_patches = 196, embed_dim = 768, 
         to_size = (8, 8, 3), num_classes = 1000):
    
    torch.manual_seed(0)
    device = torch.device(f'cpu')

    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


    dataset = ImageNetDataset(to_size = to_size, num_patches = num_patches, transform = transform)


    model = get_model(model_type, num_classes, num_patches, embed_dim, to_size)
    model = model.to(device)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

    model = learn(model = model, dataloader = dataloader,
                num_epochs = 200, optimizer = optimizer, 
                criterion = criterion, scheduler = scheduler,
                device = device)


In [36]:
train(model_type = 'vit_base_patch16_224',
         batch_size = 128, img_size = 224, num_patches = 196, embed_dim = 768, 
         to_size = (8, 8, 3), num_classes = 1000)

/var/folders/fy/p355vwwd683668y784t68b2c0000gn/T/ipykernel_77907/2643402921.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img, dtype=torch.float32)
[W NNPACK.cpp:61] Could not initialize NNPACK! Reason: Unsupported hardware.


**(a)** 
```python
x < 10      # x 小于10
```

**(b)** 
```python
x >= 10     # x 大于等于10
``` 

**(c)** 
```python
x == 10     # x 等于10
```

**(d)**
```python
x <= 10    # x 小于等于10
```

```python
A and B and C == True
```

Which of the following will evaluate to `true`  if boolean expressions `A` , `B`, and `C` are all `false`?

(A) `A = True, B = False, C = True`

(B) `A = True, B = True, C = True`

(C) `A = True, B = True, C = True`

(D) `A = False, B = True, C = True`

